## Ingest Data

I should formalize this via a function

In [2]:
import os
import pandas as pd

# collect all child files and join
files = os.listdir("../data")
files = [f for f in files if "fam" in f]

frames = []
for f in files:
    df = pd.read_csv(f"../data/{f}")
    frames.append(df)

fulldata = pd.concat(frames)

# "Counts less than 5 are masked with an asterisk (*) to protect the confidentiality of individuals in this report."
# will mark these as 0
impute_val = 2.5
fulldata = fulldata.replace(to_replace="*", value=impute_val)

# set date as index
fulldata["ReportingDate"] = pd.to_datetime(fulldata["ReportingDate"])
fulldata.set_index("ReportingDate", inplace=True)

fulldata = fulldata.apply(pd.to_numeric)

fulldata.sample(5).head()

,ZIPCode,ReasonCC_Emp,ReasonCC_Ed,ReasonCC_EmpEd,ReasonCC_PS,ReasonCC_FedEmEmp,ReasonCC_FedEmEd,ReasonCC_FedEmEmpEd,ReasonCC_FedEmPS,AvgFamilySize,...,PrimLang_3,PrimLang_4,PrimLang_5,PrimLang_6,PrimLang_7,PrimLang_8,PrimLang_9,PrimLang_10,PrimLang_11,PrimLang_12
ReportingDate,,,,,,,,,,,,,,,,,,,,,
2022-04-01,75050,96.0,2.5,5.0,2.5,2.5,2.5,10.0,10.0,3,...,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5
2021-11-01,75222,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2,...,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5
2024-03-01,78185,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,4,...,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5
2023-06-01,79360,17.0,2.5,2.5,2.5,2.5,2.5,2.5,2.5,4,...,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5
2022-01-01,75554,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,3,...,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5
